In [ ]:
#我们可以先使用线性规划方法求解整数规划问题的松弛线性规划问题，
#然后根据线性规划的结果来确定整数规划问题的整数变量的上下界，
#再使用分支定界法进行求解整数规划问题。下面是完整的函数实现：

In [1]:
from scipy.optimize import linprog

def branch_and_bound_integer_programming(c, A, b, integer_indices):
    """
    分支定界法求解整数规划问题

    参数:
        c (list): 目标函数的系数向量
        A (list): 约束条件的系数矩阵
        b (list): 约束条件的右侧常数向量
        integer_indices (list): 整数变量的索引列表

    返回:
        best_objective (float): 最优目标函数值
        best_solution (list): 最优解，即使没有找到整数解，也返回一个最接近整数解的非整数解
    """
    def integer_solution(x):
        return all(abs(x[i] - round(x[i])) < 1e-6 for i in integer_indices)

    def branch_and_bound(c, A, b, integer_indices, integer_bounds, current_objective, current_solution):
        nonlocal best_objective, best_solution

        # 判断是否为整数解
        if integer_solution(current_solution):
            # 更新最优解
            if current_objective > best_objective:
                best_objective = current_objective
                best_solution = current_solution
            return

        # 选择一个整数变量进行分支
        branch_index = None
        for idx in integer_indices:
            if abs(current_solution[idx] - round(current_solution[idx])) >= 1e-6:
                branch_index = idx
                break

        # 如果所有整数变量都已经是整数解，则说明问题无解
        if branch_index is None:
            return

        # 对当前分支变量的两个取值进行递归分支
        lower_bound, upper_bound = integer_bounds[branch_index]
        branch_value = round(current_solution[branch_index])

        # 分支：当前分支变量取下界
        A_branch = A.copy()
        b_branch = b.copy()
        A_branch.append([0] * len(c))
        A_branch[-1][branch_index] = 1
        b_branch.append(lower_bound)
        lp_result = linprog(c, A_ub=A_branch, b_ub=b_branch)
        if lp_result.success and lp_result.fun > best_objective:
            branch_and_bound(c, A_branch, b_branch, integer_indices, integer_bounds,
                             lp_result.fun, lp_result.x)

        # 分支：当前分支变量取上界
        A_branch = A.copy()
        b_branch = b.copy()
        A_branch.append([0] * len(c))
        A_branch[-1][branch_index] = -1
        b_branch.append(-upper_bound)
        lp_result = linprog(c, A_ub=A_branch, b_ub=b_branch)
        if lp_result.success and lp_result.fun > best_objective:
            branch_and_bound(c, A_branch, b_branch, integer_indices, integer_bounds,
                             lp_result.fun, lp_result.x)

    best_objective = float('-inf')
    best_solution = None

    # 先使用线性规划方法求解整数规划问题的松弛线性规划问题
    lp_result = linprog(c, A_ub=A, b_ub=b)
    if lp_result.success:
        # 根据线性规划结果来确定整数规划问题的整数变量的上下界
        integer_bounds = [(int(lp_result.x[i]), int(lp_result.x[i])) if i in integer_indices else (0, None) for i in range(len(c))]

        # 使用分支定界法求解整数规划问题
        branch_and_bound(c, A, b, integer_indices, integer_bounds, lp_result.fun, lp_result.x)

    return best_objective, best_solution

# 测试例子
c = [-2, -3]
A = [[3, 2], [1, 4]]
b = [12, 8]
integer_indices = [0, 1]

# 使用分支定界法求解整数规划问题
best_objective, best_solution = branch_and_bound_integer_programming(c, A, b, integer_indices)

print("最优目标函数值：", best_objective)
print("最优解：", best_solution)

最优目标函数值： -9.0
最优解： [3. 1.]
